In [1]:
import json
from neo4j import GraphDatabase
import pandas as pd

In [2]:
#Load dataset
with open('../building-dataset/json/processed_data_mvd.json', 'r') as file:
    data = json.load(file)

In [3]:
#Load dataset
with open('../building-dataset/json/docs_only_mvd_clean.json', 'r') as file:
    docs_data = json.load(file)

In [29]:
#Use for global database connection
driver = GraphDatabase.driver(uri="bolt://3.91.134.249:7687", auth=("neo4j", "escapes-certification-array"))

In [4]:
#use for local database connection
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "password"))

In [5]:
#Convert JSON to DataFrame
df = pd.DataFrame(data)

In [31]:
document_frame = pd.DataFrame(docs_data)

In [32]:
df[["labels", "document_id", "title", "normalized_name", "parent_id"]]

,labels,document_id,title,normalized_name,parent_id
0,COURT,8,Satish Kumar & Anr vs State & Anr on 25 Septem...,Rajasthan High Court - Jodhpur,ef1bc19969df4a6fbfa1195f5d97742d
1,PRECEDENT,8,Satish Kumar & Anr vs State & Anr on 25 Septem...,Satish Kumar & Anr vs State & Anr,ef1bc19969df4a6fbfa1195f5d97742d
2,DATE,8,Satish Kumar & Anr vs State & Anr on 25 Septem...,"25 September, 2008",ef1bc19969df4a6fbfa1195f5d97742d
3,PRECEDENT,8,Satish Kumar & Anr vs State & Anr on 25 Septem...,S. B. Criminal Misc. Petition No.49/2008,ef1bc19969df4a6fbfa1195f5d97742d
4,PRECEDENT,8,Satish Kumar & Anr vs State & Anr on 25 Septem...,Satish Kumar & Another. V. State Of Rajasthan ...,ef1bc19969df4a6fbfa1195f5d97742d
...,...,...,...,...,...
1535,RESPONDENT,4,Abdul Mannan Prodhani vs The State Of Assam An...,Sahina Begum Mazumdar,8829c554a4cc4b7cac11f300a0b44255
1536,RESPONDENT,4,Abdul Mannan Prodhani vs The State Of Assam An...,Sudarshan Bhattarai,8829c554a4cc4b7cac11f300a0b44255
1537,LAWYER,4,Abdul Mannan Prodhani vs The State Of Assam An...,S M Abdullah,8829c554a4cc4b7cac11f300a0b44255
1538,JUDGE,4,Abdul Mannan Prodhani vs The State Of Assam An...,Manash Ranjan Pathak,8829c554a4cc4b7cac11f300a0b44255


In [33]:
df.columns

Index(['id', 'normalized_name', 'start', 'end', 'text', 'labels',
       'document_id', 'parent_id', 'publishdate', 'title', 'docsource'],
      dtype='object')

In [34]:
#Group documents by ID
grouped = df.groupby('document_id')

In [35]:
#Method to print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total:
        print()

In [36]:
#gets sub-text map
#subtextID: text
documentIDToSentences = {}
#case name : set()

for name, group in grouped:
    group = group.sample(frac=1)

    doc_id = group.iloc[0]['document_id']
    documentIDToSentences[doc_id] = set()

    for index, row in group.iterrows():
        parentId = row['parent_id']
        text = row['text']

        subSentenceObject = {
            "text": text,
            "parentId": parentId
        }

        documentIDToSentences[doc_id].add(json.dumps(subSentenceObject))

documentIDToSentences

{1: {'{"text": "And in this connection, they would draw attention to the Bombay Collectorate Trade Notice No. 57/1987 dated 7-8-1987.", "parentId": "850bf9c517e34f418de9787f8e561917"}',
  '{"text": "Customs, Excise and Gold Tribunal - Delhi\\nAdarsh Packers (P) Ltd. vs Collector Of Central Excise on 22 March, 1994Equivalent citations: 1994 (71) Elt 837 Tri Del\\nOrder\\nS.K. Bhatnagar, Vice President\\n1.  This is a Stay Application filed with reference to the Order-in-Appeal of Collector of Central Excise (Appeals) dated 30-11-1993.\\n2.", "parentId": "b643e0bb8e924b21b519d9bd975ef2ff"}'},
 2: {'{"text": "Assuming, however, that \\"drugs\\" and \\"substances\\" in Section 2(g) of the Act have to be mutually exclusive, the definition of \\"narcotic\\" in Section 2(h) makes it abundantly clear that a \\"narcotic\\" is a \\"substance\\", if not a \\"drug\\".", "parentId": "ea4cefeb62574a8ea12f6a944244760a"}',
  '{"text": "Counsel for the respondent, on the other hand, contended that the 

In [42]:
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

Failed to read from defunct connection IPv4Address(('3.91.134.249', 7687)) (IPv4Address(('3.91.134.249', 7687)))


ServiceUnavailable: Failed to read from defunct connection IPv4Address(('3.91.134.249', 7687)) (IPv4Address(('3.91.134.249', 7687)))

In [38]:
#Add case nodes to database
count = 0
total = len(grouped)
print("CREATING CASE NODES: ")
printProgressBar(0, total, prefix='Progress:', suffix='Complete', length=total)

with driver.session() as session:
    for name, group in grouped:
        group = group.sample(frac=1)

        caseName = group.iloc[0]['title']
        caseId = group.iloc[0]['document_id']

        document_by_id = document_frame.loc[document_frame['tid'] == caseId]['doc']

        session.run("CREATE (n:Case {name: $name, type: $type, case_id: $case_id, document: $document})", name=caseName, type='CASE', case_id=caseId, document=document_by_id)

        count = count + 1
        printProgressBar(count, total, prefix='Progress:', suffix='Complete', length=total)

CREATING CASE NODES: 
Progress: |███████████████████████████████████████████████| 100.0% Complete


In [40]:
#Add case sub-text nodes to database and add relationship between them
count = 0
total = len(grouped)
print("CONNECTING SUB-TEXT TO CASE NODES")
printProgressBar(0, total, prefix='Progress:', suffix='Complete', length=total)

with driver.session() as session:
    for name, group in grouped:
        group = group.sample(frac=1)

        caseId = group.iloc[0]['document_id']
        caseSubTextObjects = documentIDToSentences[caseId]
        for subText in caseSubTextObjects:
            text = json.loads(subText)['text']
            parentId = json.loads(subText)['parentId']

            if session.run(
                    "MATCH (n:SubText {name: $parent_id, text: $text, type: $type, parent_id: $parent_id, case_id: $case_id}) RETURN n",
                    text=text,
                    type='SUB-TEXT', parent_id=parentId, case_id=caseId).single() is None:

                session.run("CREATE (n:SubText {name: $text, type: $type, parent_id: $parent_id, case_id: $case_id})",
                            text=text,
                            type='SUB-TEXT', parent_id=parentId, case_id=caseId)

                session.run("MATCH (a:SubText),(b:Case) WHERE a.case_id = $case_id AND b.case_id = "
                            "$case_id AND b.type = 'CASE' AND a.parent_id = $parent_id "
                            "MERGE (a)-[ r:SUBTEXT_OF {relationship: $relationship}]->(b)",
                            case_id=caseId,
                            relationship="Sub-Text with ID: " + str(parentId) + ", of case with ID " + str(caseId), parent_id=parentId)

        count = count + 1
        printProgressBar(count, total, prefix='Progress:', suffix='Complete', length=total)

CONNECTING SUB-TEXT TO CASE NODES


KeyboardInterrupt: 

In [41]:
#Add end nodes to the database and connect them to sub-text nodes
count = 0
total = len(grouped)
print("CONNECTING END-NODES TO CASE NODES")
printProgressBar(0, total, prefix='Progress:', suffix='Complete', length=total)

with driver.session() as session:
    for name, group in grouped:
        group = group.sample(frac=1)

        caseName = group.iloc[0]['title']
        caseId = group.iloc[0]['document_id']

        for index, row in group.iterrows():
            normalizedName = row['normalized_name'].upper()
            label = row['labels']
            parent_id = row['parent_id']

            if session.run("MATCH (n:Entity {name: $name, type: $type}) RETURN n", name=normalizedName,
                           type=label).single() is None:

                session.run("CREATE (n:Entity {name: $name, type: $type, parent_id: $parent_id})", name=normalizedName,
                            type=label, parent_id=[parent_id])

            else:
                session.run("MATCH (n:Entity {name: $name, type: $type}) SET n.parent_id = apoc.coll.union(n.parent_id, "
                            "$parent_id)", name=normalizedName,
                            type=label, parent_id=[parent_id])

            relationship = "Normalized Name found in subtext: " + str(parent_id) + ", of case with ID " + str(caseId)

            session.run("MATCH (a:Entity),(b:SubText) WHERE a.name = $end_node AND b.parent_id = $parent_id"
                        " AND b.case_id = $case_id"
                        " AND a.type = $type"
                        " CREATE (a)-[ r:" + label + "_OF" + " {relationship: $relationship}]->(b)",
                        end_node=normalizedName, parent_id=parent_id,
                        case_id=caseId,
                        relationship=relationship,
                        type=label
                        )

        count = count + 1
        printProgressBar(count, total, prefix='Progress:', suffix='Complete', length=total)

CONNECTING END-NODES TO CASE NODES


KeyboardInterrupt: 

In [6]:
#There can be multiple relationships pointing to the same node as the normalized_text can appear multiple times in the same sentence, those relations are removed.
with driver.session() as session:
    session.run("MATCH (a:Entity)-[r]->(b:SubText)"
                " WITH a, b, type(r) AS rel_type, r.name AS name, COLLECT(r) AS rels, COUNT(*) AS count"
                " WHERE count > 1"
                " UNWIND rels[1..] AS duplicate"
                " DELETE duplicate")

In [94]:
#Use the following query to get specific cases
print("MATCH (a:Entity),(b:SubText),(c:Case) WHERE c.case_id in [ids here] AND b.case_id = c.case_id AND b.parent_id in a.parent_id return a, b, c")